In [1]:
# 2
import pandas as pd
aucs = pd.read_csv("AUCs.txt", sep="\t").drop("Unnamed: 0", axis=1)
aucs.head()

,C4.5,C4.5+m,C4.5+cf,C4.5+m+cf
0,0.763,0.768,0.771,0.798
1,0.599,0.591,0.590,0.569
2,0.954,0.971,0.968,0.967
3,0.628,0.661,0.654,0.657
4,0.882,0.888,0.886,0.898


In [2]:
# 2
from scipy import stats
corr_data = []

In [4]:
most_significant = 1  # smallest p-value
pair = (-1, -1)  # indices of columns

for i, lhs_column in enumerate(aucs.columns):
    for j, rhs_column in enumerate(aucs.columns):
        if i < j:
            corr, p = stats.wilcoxon(aucs[lhs_column], aucs[rhs_column])
            corr_data.append([lhs_column, rhs_column, corr, p])
            if p < most_significant:
                most_significant = p
                pair = (i, j)
                
print "Most significant difference is between %s and %s" % (aucs.columns[pair[0]], aucs.columns[pair[1]])

Most significant difference is between C4.5 and C4.5+m


In [10]:
# 3
aucs_corr = pd.DataFrame.from_records(corr_data)
aucs_corr.columns = ['product_A', 'product_B', 'corr', 'p']
aucs_corr['reject'] = aucs_corr.p < 0.05
aucs_corr[aucs_corr["reject"]]  # 4

,product_A,product_B,corr,p,reject
0,C4.5,C4.5+m,6.5,0.010757,True
2,C4.5,C4.5+m+cf,11.0,0.015906,True
3,C4.5+m,C4.5+cf,17.0,0.046333,True
5,C4.5+cf,C4.5+m+cf,10.0,0.022909,True
6,C4.5,C4.5+m,6.5,0.010757,True
8,C4.5,C4.5+m+cf,11.0,0.015906,True
9,C4.5+m,C4.5+cf,17.0,0.046333,True
11,C4.5+cf,C4.5+m+cf,10.0,0.022909,True


In [11]:
# 5
from statsmodels.sandbox.stats.multicomp import multipletests

reject, p_corrected, a1, a2 = multipletests(aucs_corr.p, 
                                            alpha = 0.05, 
                                            method = 'holm') 
aucs_corr['p_corrected'] = p_corrected
aucs_corr['reject'] = reject

aucs_corr  # 0

,product_A,product_B,corr,p,reject,p_corrected
0,C4.5,C4.5+m,6.5,0.010757,False,0.129086
1,C4.5,C4.5+cf,43.0,0.861262,False,1.000000
2,C4.5,C4.5+m+cf,11.0,0.015906,False,0.159064
3,C4.5+m,C4.5+cf,17.0,0.046333,False,0.277996
4,C4.5+m,C4.5+m+cf,22.0,0.327826,False,1.000000
5,C4.5+cf,C4.5+m+cf,10.0,0.022909,False,0.183273
6,C4.5,C4.5+m,6.5,0.010757,False,0.129086
7,C4.5,C4.5+cf,43.0,0.861262,False,1.000000
8,C4.5,C4.5+m+cf,11.0,0.015906,False,0.159064
9,C4.5+m,C4.5+cf,17.0,0.046333,False,0.277996


In [6]:
# 6
reject, p_corrected, a1, a2 = multipletests(aucs_corr.p, 
                                            alpha = 0.05, 
                                            method = 'fdr_bh') 
aucs_corr['p_corrected'] = p_corrected
aucs_corr['reject'] = reject

aucs_corr  # 3

,product_A,product_B,corr,p,reject,p_corrected
0,C4.5,C4.5+m,6.5,0.010757,True,0.045818
1,C4.5,C4.5+cf,43.0,0.861262,False,0.861262
2,C4.5,C4.5+m+cf,11.0,0.015906,True,0.045818
3,C4.5+m,C4.5+cf,17.0,0.046333,False,0.069499
4,C4.5+m,C4.5+m+cf,22.0,0.327826,False,0.393391
5,C4.5+cf,C4.5+m+cf,10.0,0.022909,True,0.045818
